In [1]:
pip install transformers

     |████████████████████████████████| 3.4 MB 5.3 MB/s 
     |████████████████████████████████| 895 kB 49.5 MB/s 
     |████████████████████████████████| 3.3 MB 19.1 MB/s 
     |████████████████████████████████| 596 kB 73.0 MB/s 
     |████████████████████████████████| 61 kB 512 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
pip install tflearn

     |████████████████████████████████| 107 kB 5.4 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=f4677c9759c7e2b02e0c74232360c2d7ed4924c29a9b0559303ba654e8734346
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn


In [194]:
import re
import io
import random
import string
import warnings
import numpy as np
import tensorflow as tf
import json
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd


import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer
import tensorflow as tf
nltk.download('popular', quiet=True) # for downloading packages

True

In [224]:
with open('data.json') as fin:
    data = json.load(fin)

In [225]:
#Tokenisation
contractions_dict = { "ain’t": "are not", "’s":" is", "aren’t": "are not", "can’t": "cannot", "can’t’ve": "cannot have", "‘cause": "because", "could’ve": "could have", "couldn’t": "could not", "couldn’t’ve": "could not have", "didn’t": "did not", "doesn’t": "does not", "don’t": "do not", "hadn’t": "had not", "hadn’t’ve": "had not have", "hasn’t": "has not", "haven’t": "have not", "he’d": "he would", "he’d’ve": "he would have", "he’ll": "he will", "he’ll’ve": "he will have", "how’d": "how did", "how’d’y": "how do you", "how’ll": "how will", "I’d": "I would", "I’d’ve": "I would have", "I’ll": "I will", "I’ll’ve": "I will have", "I’m": "I am", "I’ve": "I have", "isn’t": "is not", "it’d": "it would", "it’d’ve": "it would have", "it’ll": "it will", "it’ll’ve": "it will have", "let’s": "let us", "ma’am": "madam", "mayn’t": "may not", "might’ve": "might have", "mightn’t": "might not", "mightn’t’ve": "might not have", "must’ve": "must have", "mustn’t": "must not", "mustn’t’ve": "must not have", "needn’t": "need not", "needn’t’ve": "need not have", "o’clock": "of the clock", "oughtn’t": "ought not", "oughtn’t’ve": "ought not have", "shan’t": "shall not", "sha’n’t": "shall not", "shan’t’ve": "shall not have", "she’d": "she would", "she’d’ve": "she would have", "she’ll": "she will", "she’ll’ve": "she will have", "should’ve": "should have", "shouldn’t": "should not", "shouldn’t’ve": "should not have", "so’ve": "so have", "that’d": "that would", "that’d’ve": "that would have", "there’d": "there would", "there’d’ve": "there would have", "they’d": "they would", "they’d’ve": "they would have","they’ll": "they will",
 "they’ll’ve": "they will have", "they’re": "they are", "they’ve": "they have", "to’ve": "to have", "wasn’t": "was not", "we’d": "we would", "we’d’ve": "we would have", "we’ll": "we will", "we’ll’ve": "we will have", "we’re": "we are", "we’ve": "we have", "weren’t": "were not","what’ll": "what will", "what’ll’ve": "what will have", "what’re": "what are", "what’ve": "what have", "when’ve": "when have", "where’d": "where did", "where’ve": "where have", "i'm" : "i am","there's":"there is","i've":"i have","what's":"what is","can't":"can not","don't":"do not","i'll":"i will","we're":"we are","it's":"it is","yr":"year",
 "who’ll": "who will", "who’ll’ve": "who will have", "who’ve": "who have", "why’ve": "why have", "will’ve": "will have", "won’t": "will not", "won’t’ve": "will not have", "would’ve": "would have", "wouldn’t": "would not", "wouldn’t’ve": "would not have", "y’all": "you all", "y’all’d": "you all would", "y’all’d’ve": "you all would have", "y’all’re": "you all are", "y’all’ve": "you all have", "you’d": "you would", "you’d’ve": "you would have", "you’ll": "you will", "you’ll’ve": "you will have", "you’re": "you are", "you’ve": "you have"}

contractions_re = re.compile('(%s)'%'|'.join(contractions_dict.keys()))
def expand_contractions(s, contractions_dict=contractions_dict):
  def replace(match):
    return contractions_dict[match.group(0)]
  return contractions_re.sub(replace, s)


def preprocess(sentences):
    sentences  = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",sentences).split())
    sentences  = sentences.replace('https?:\/\/.*[\r\n]*', '')
    # sentences  = sentences.normalize('NFKD').encode('ascii', errors='ignore').decode('utf-8')
    sentences  = sentences.replace('\d+', '')
    sentences  = sentences.replace('[^\w\s]', '')
    sentences  = expand_contractions(sentences)
  

In [226]:
words = []
sentences = []
labels = []
docs_x = []
docs_y = []
bert_inputs = []
X = []
y = []
labels = []

In [227]:
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
lemmer = WordNetLemmatizer()
stemmer = LancasterStemmer()

def StemTokens(tokens):
    return [stemmer.stem(token.lower()) for token in tokens]
def LemTokens(tokens):
    return [lemmer.lemmatize(token.lower()) for token in tokens]
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [228]:
for intent in data['intents']:
      for pattern in intent['patterns']:
            preprocess(pattern)
            tokenize = LemNormalize(pattern)
            # tokenize = StemTokens(tokenize)
            words.extend(tokenize)
            sentences.append(pattern)
            docs_x.append(tokenize)
            docs_y.append(intent["tag"])

      labels.append(intent["tag"])


In [230]:
words = LemTokens(words)
words = sorted(list(set(words)))

labels = sorted(labels)

X = []
y = []

init = [0]*(len(labels))

for x, doc in enumerate(docs_x):
      bag = []
      find_words = LemTokens(doc)
      
      for w in words:
            if w in find_words:
                  bag.append(1)
            else:
                  bag.append(0)
                  
      output_row = init[:]
      output_row[labels.index(docs_y[x])] = 1
      X.append(bag)
      y.append(output_row)

In [231]:
training = np.array(X)
output = np.array(y)

### Alternative shortcut using built in tfidf vectorizer and label encoder from sklearn

In [256]:
df = pd.DataFrame(data={'respond':sentences,'intent':docs_y})

In [279]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

In [258]:
vectorizer = TfidfVectorizer()
tfidf_dict = vectorizer.fit_transform(df['respond'])

In [259]:
lb = LabelEncoder()
y = lb.fit_transform(df['intent'])
dummy_y_train = np_utils.to_categorical(y)

In [254]:
# df = pd.concat([df,pd.get_dummies(df['intent'],drop_first=True)],axis=1).drop('intent',axis=1)

# Modelling

## Baseline with Multinomial Naive Bayes

In [261]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer,CountVectorizer
from sklearn.naive_bayes import MultinomialNB

text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])

In [263]:
text_clf.fit(df['respond'],df['intent'])

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [265]:
text_clf.predict(['whats your name'])

array(['services'], dtype='<U10')

## with DNN tflearn 

In [273]:
import tflearn

tf.compat.v1.reset_default_graph()
net = tflearn.input_data(shape=[None,len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

In [274]:
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)

Training Step: 5999  | total loss: 0.42983 | time: 0.029s
| Adam | epoch: 1000 | loss: 0.42983 - acc: 0.9654 -- iter: 40/44
Training Step: 6000  | total loss: 0.39294 | time: 0.032s
| Adam | epoch: 1000 | loss: 0.39294 - acc: 0.9688 -- iter: 44/44
--


In [275]:
def bag_of_words(s, words):
    bag = [0]*len(words)
    s_words = LemNormalize(s)
    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return np.array([bag])

In [276]:
pred = model.predict(bag_of_words('whats your name',words))

In [277]:
pred[0]

array([2.8029143e-09, 2.4803526e-07, 2.5656888e-10, 6.8185903e-04,
       2.5183486e-03, 8.1319413e-05, 8.6423178e-04, 9.2904055e-03,
       9.8648316e-01, 2.5494082e-08, 8.0473248e-05], dtype=float32)

In [278]:
labels[np.argmax(pred)]

'name'

In [154]:
model.save('model.tflearn')

INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


## with DNN Keras

In [173]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("keras_model.h5",save_best_weights=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

In [176]:
def model(n_layers=2,n_neurons=8,input_shape=tfidf_dict.shape[1]):
      model  = tf.keras.models.Sequential()
      model.add(tf.keras.layers.Dense(tfidf_dict.shape[0],input_dim = input_shape, activation='elu'))
      for _ in range(n_layers):
        model.add(tf.keras.layers.Dense(n_neurons,kernel_initializer='he_normal',activation='elu'))
      model.add(tf.keras.layers.Dense(len(labels),activation='softmax'))
      model.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.Nadam(),metrics=['accuracy'])
      return model
    
model = model()

In [103]:
model.fit(tfidf_dict,np.array([np.argmax(x) for x in output]),batch_size=5,epochs=800,callbacks=[checkpoint_cb,early_stopping_cb])

Train on 44 samples
Epoch 1/800
44/44 [==============================] - 0s 5ms/sample - loss: 2.3876 - acc: 0.0909
Epoch 2/800
44/44 [==============================] - 0s 997us/sample - loss: 2.3584 - acc: 0.1818
Epoch 3/800
44/44 [==============================] - 0s 857us/sample - loss: 2.3236 - acc: 0.1818
Epoch 4/800
44/44 [==============================] - 0s 1ms/sample - loss: 2.2881 - acc: 0.2273
Epoch 5/800
44/44 [==============================] - 0s 930us/sample - loss: 2.2493 - acc: 0.3182
Epoch 6/800
44/44 [==============================] - 0s 953us/sample - loss: 2.2106 - acc: 0.3864
Epoch 7/800
44/44 [==============================] - 0s 947us/sample - loss: 2.1689 - acc: 0.4091
Epoch 8/800
44/44 [==============================] - 0s 1ms/sample - loss: 2.1260 - acc: 0.4545
Epoch 9/800
44/44 [==============================] - 0s 947us/sample - loss: 2.0806 - acc: 0.5000
Epoch 10/800
44/44 [==============================] - 0s 934us/sample - loss: 2.0353 - acc: 0.5000
Epoch

In [177]:
def tfidf_form(s):
    s = pd.DataFrame(data={'respond':[s]})
    s['respond'].apply(lambda x: LemNormalize(x))
    return vectorizer.transform(s['respond'])

In [ ]:
model = tf.keras.models.load_model('keras_model.h5')
pred = model.predict(tfidf_form('how old are you? im curious'))

In [250]:
labels[np.argmax(pred)]

'age'